In [1]:
import requests
import cnfg
import json
import pickle
import time
from datetime import datetime
from collections import defaultdict
from urlparse import urlparse
from pymongo import MongoClient

In [2]:
config = cnfg.load(".genius_config")
client_key = config["client_key"]
client_secret = config["client_secret"]
access_token = 'uK247ank6jF7DY_E7TEMIdbuhT2IBVsVeFXh6AluxswFUfrlFNfXNu2pUFPrGv2c'
auth = 'Bearer '+access_token
req_start = 'https://api.genius.com'

In [70]:
client = MongoClient()
db = client.music
s = db.songs
# counter
c = s.find()

In [4]:
def pickleLoad(filename):
    with open(filename, 'rb') as f:
        data = pickle.load(f) 
    return data

def pickleDump(data, filename):
    with open(filename, 'wb') as f:
        pickle.dump(data, f)

In [5]:
def api_call(validurl):
    call = validurl
    r = requests.get(call, headers={'Authorization':auth})
    if r.status_code==200:
        return json.loads(r.content)
    else:
        print r.content
        return r.content

In [7]:
songs = [653082, 2396122, 2330692, 2348651, 2295548, 2425602, 2165110, 2312297, 2394578, \
         2305085, 2422646, 2307164, 709256, 2417517, 2401700, 2413367, 2342860, 2439141, \
         2347598, 2432858, 2396122, 2411389, 2433137, 2330692, 2348651, 2295548, 2372618, \
         2425602, 2165110, 2312297, 2394578, 2305085, 2422646, 2307164, 1783676, 709256, \
         2417517, 2401700, 2426609]

In [11]:
artists = list(s.find({}, {'artist_id':1, '_id':0}))

In [15]:
len(artists)

226

In [17]:
artists = [int(x['artist_id']) for x in artists]

In [20]:
writers = list(s.find({}, {'writers':1, '_id':0}))

In [27]:
writers[0]

{u'writers': [{u'artist': u'Remy Boyz', u'id': 211094}]}

In [25]:
len(writers)

232

In [31]:
for writer in writers:
    if writer:
        for w in writer['writers']:
            artists.append(int(w['id']))

In [33]:
featured_artists = list(s.find({}, {'featured_artists':1, '_id':0}))

In [37]:
for fa in featured_artists:
    if fa:
        for w in fa['featured_artists']:
            artists.append(int(w['id']))

In [39]:
producers = list(s.find({}, {'producers':1, '_id':0}))
for p in producers:
    if p:
        for w in p['producers']:
            artists.append(int(w['id']))

In [46]:
desc_refs = list(s.find({}, {'description_refs.artist':1, 'description_refs.song':1, '_id':0}))

In [67]:
for dr in desc_refs:
    if 'artist' in dr['description_refs']:
        a = dr['description_refs']['artist']
        for artist in a:
            artists.append(int(artist['id']))
    if 'song' in dr['description_refs']:
        pass
#         print len(dr['description_refs']['song'])

In [55]:
anno_refs = list(s.find({}, {'references.artist':1, 'references.song':1, '_id':0}))

In [61]:
for ar in anno_refs:
    if 'artist' in ar['references']:
        a = ar['references']['artist']
        for artist in a:
            artists.append(int(artist['id']))
    if 'song' in ar['references']:
        pass

In [71]:
sampled_songs = list(s.find({}, {'sampled_songs':1, '_id':0}))

In [96]:
for samp in sampled_songs:
    if samp:
        for w in samp:
            for song in samp[w]:
                artists.append(song['primary_artist']['id'])

In [83]:
len(artists)

1157

In [84]:
artists = list(set(artists))

In [85]:
len(artists)

515

In [172]:
first_song = s.find({'id':653082}, {'_id':0, 'description_refs.artist':1, 'description_refs.song':1, \
                       'sampled_songs':1, 'producers':1, 'writers':1, \
                       'featured_artists':1, 'artist_id':1, 'id':1}).next()

In [173]:
first_song

{u'artist_id': 216609.0,
 u'description_refs': {u'song': [{u'artist': u'Fetty Wap',
    u'id': u'496445',
    u'title': u'Trap Queen'}]},
 u'featured_artists': [{u'artist': u'Monty', u'id': 19635}],
 u'id': 653082,
 u'producers': [{u'artist': u'JayFrance', u'id': 557945},
  {u'artist': u'Nick EBeats', u'id': 72396}],
 u'writers': [{u'artist': u'Remy Boyz', u'id': 211094}]}

In [160]:
weights = {'artist_id': 10., 'producers':10., 'writers': 10., 'featured_artists': 8., \
          'sampled_songs': 7., 'description_refs': 5., 'references': 4.}

In [144]:
contributors = []
cont_dict = defaultdict(list)
for e in first_song:
    if e == 'artist_id':
        contributors.append('artist_id')
        cont_dict['artist_id'].append(int(first_song[e]))
        print int(first_song[e]), " primary artist id"
    if e == 'description_refs':
        if 'artist' in e:
            contributors.append('description_refs')
            for a in e['artist']:
                cont_dict['description_refs'].append(int(a['id']))
                print int(a['id']), " description_refs"
    if e == 'references' in first_song:
        if 'artist' in e:
            contributors.append('references')
            for a in e['artist']:
                cont_dict['references'].append(int(a['id']))
                print int(a['id']), " references"
    if e == 'producers':
        contributors.append('producers')
        for a in first_song[e]:
            cont_dict['producers'].append(int(a['id']))
            print int(a['id']), " producer"
    if e == 'writers':
        contributors.append('writers')
        for a in first_song[e]:
            cont_dict['writers'].append(int(a['id']))
            print int(a['id']), " writer"
    if e == 'featured_artists':
        contributors.append('featured_artists')
        for a in first_song[e]:
            cont_dict['featured_artists'].append(int(a['id']))
            print int(a['id']), " featured_artist"
    if e == 'sampled_songs':
        contributors.append('sampled_songs')
        for samp in first_song[e]:
            for sampsong in samp:
                cont_dict['sampled_songs'].append(int(sampsong['primary_artist']['id']))
                print int(sampsong['primary_artist']['id']), " sampled song"

557945  producer
72396  producer
216609  primary artist id
211094  writer
19635  featured_artist


In [133]:
contributors

['producers', 'artist_id', 'writers', 'featured_artists']

In [145]:
cont_dict

defaultdict(list,
            {'artist_id': [216609],
             'featured_artists': [19635],
             'producers': [557945, 72396],
             'writers': [211094]})

In [152]:
total = 0
for artist_type in contributors:
    total += weights[artist_type]

In [153]:
new_total = 0
for artist_type in cont_dict:
    print artist_type
    print weights[artist_type]
    new_total += weights[artist_type]

producers
10
artist_id
10
writers
10
featured_artists
8


In [154]:
new_total

38

In [155]:
total

38

In [137]:
artists = [int(x) for x in artists]

In [139]:
len(artists)

515

In [140]:
artists = list(set(artists))

In [141]:
len(artists)

451

In [142]:
pickleDump(artists, 'artists_in_db.pkl')

In [167]:
weighted_artists = defaultdict(float)

In [168]:
for x, y in cont_dict.iteritems():
    frac = weights[x]/len(y)
    for influencer in y:
        weighted_artists[influencer] = frac/total

In [169]:
weighted_artists

defaultdict(float,
            {19635: 0.21052631578947367,
             72396: 0.13157894736842105,
             211094: 0.2631578947368421,
             216609: 0.2631578947368421,
             557945: 0.13157894736842105})

In [171]:
songsum = 0
for key, value in weighted_artists.iteritems():
    songsum += value
print songsum

1.0


In [164]:
import pandas as pd

In [165]:
df = pd.DataFrame(columns=artists)

In [206]:
songid = first_song['id']

,1,2,3,4,2057,329739,13,589827,20503,589828,...,38842,1985,99318,39585,28651,1362,2035,14325,626678,64581


In [181]:
blah = {(a):(0 if a not in weighted_artists.keys() else weighted_artists[a]) for a in artists}

1.0

In [191]:
first_song_series = pd.Series(data = blah.values(), index = blah.keys())

In [200]:
df = df.append(first_song_series, ignore_index=True)

In [201]:
df[1]

0    0
Name: 1, dtype: float64

In [202]:
df[19635]

0    0.210526
Name: 19635, dtype: float64

In [208]:
df['myindex'] = [19635]

In [210]:
df['myindex']

0    19635
Name: myindex, dtype: int64

In [211]:
df.set_index('myindex', inplace=True, drop=True)

In [212]:
df

,1,2,3,4,2057,329739,13,589827,20503,589828,...,38842,1985,99318,39585,28651,1362,2035,14325,626678,64581
myindex,,,,,,,,,,,,,,,,,,,,,
19635,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [213]:
#### reverse the order tomorrow!!!!